In [3]:
from google.cloud import bigquery

client = bigquery.Client(project="valued-lambda-478823-h0")

query = """
SELECT
*
FROM `valued-lambda-478823-h0.azdd_test.gender_status_clean`
"""


df = client.query(query).to_dataframe()

df.head()

,county,active_female,active_male,active_unknown,active_total,inactive_female,inactive_male,inactive_unknown,inactive_total,prereg_female,prereg_male,prereg_unknown,prereg_total,grand_total
0,Cheyenne,613,611,0,1224,67,86,0,153,9,7,0,16,1393
1,Hinsdale,321,351,0,672,30,39,0,69,2,2,0,4,745
2,Mineral,372,359,0,731,48,56,0,104,0,2,0,2,837
3,San Juan,270,323,0,593,45,57,0,102,1,0,0,1,696
4,Baca,1296,1254,1,2551,160,151,0,311,26,16,0,42,2904


In [6]:
id_cols = ["county"]

value_cols = [
    c for c in df.columns
    if c not in id_cols
    and not c.endswith("_total")
    and c != "grand_total"
]

long_df = df.melt(
    id_vars=id_cols,
    value_vars=value_cols,
    var_name="status_gender",
    value_name="count"
)

long_df[["status", "gender"]] = long_df["status_gender"].str.split("_", n=1, expand=True)

long_df = long_df[["county", "status", "gender", "count"]]

long_df.head()

,county,status,gender,count
0,Cheyenne,active,female,613
1,Hinsdale,active,female,321
2,Mineral,active,female,372
3,San Juan,active,female,270
4,Baca,active,female,1296


In [7]:
from google.cloud import bigquery

project = "valued-lambda-478823-h0"
dataset = "azdd_test"
table = f"{project}.{dataset}.gender_status_long"

client = bigquery.Client(project=project)

job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE"
)

load_job = client.load_table_from_dataframe(
    long_df,
    destination=table,
    job_config=job_config
)

load_job.result()

print(f"Loaded {client.get_table(table).num_rows} rows into {table}")

Loaded 585 rows into valued-lambda-478823-h0.azdd_test.gender_status_long
